# ***Hello!Wlecome to my notebook*** 

# ***Import libraries and model"s***

In [ ]:
from tensorflow.keras.applications import VGG16, VGG19, ResNet50, ResNet50V2

# ***Architectures VGG16***

In [ ]:
model = VGG16()

In [ ]:
model.summary()

# ***Architectures VGG19***

In [ ]:
model2 = VGG19()

In [ ]:
model2.summary()

# ***Architectures ResNet50***

In [ ]:
model3 = ResNet50()

In [ ]:
model3.summary()

In [ ]:
import numpy as np
import pandas as pd
import os

import tensorflow as tf
from keras.models import *
from keras.layers import *

from base64 import b64encode
from IPython.display import HTML
import cv2
import math

from skimage.io import imread
from PIL import Image
import imgaug.augmenters as iaa
from tqdm import tqdm

from sklearn.model_selection import StratifiedShuffleSplit

# ****Reading Dataset****

In [ ]:
videos_directory = '/kaggle/input/real-life-violence-situations-dataset'

In [ ]:
imagesize = 128
colorchnnels = 3

# ***displaying Sample from Data set***

In [ ]:
def playingvideos(filename):
    html = ''
    video = open(filename, 'rb').read()
    src = 'data:video/mp4;base64,' + b64encode(video).decode()
    html += '<video width=640 muted controls autoplay loop><source src="%s" type="video/mp4"></video>' % src 
    return HTML(html)

In [ ]:
playingvideos('/kaggle/input/real-life-violence-situations-dataset/Real Life Violence Dataset/NonViolence/NV_12.mp4')

# ***Preprocessing and Augmentation***

In [ ]:
def convert_video_to_frames(video):
    #to capture the video
    videocaptured = cv2.VideoCapture(video)
    rate = math.floor(videocaptured.get(3))
    
    count = 0
    frames = []
    
    while videocaptured.isOpened():
        ID = videocaptured.get(1)
        sucess, image = videocaptured.read()
        
        if sucess:
            if (ID % 7 == 0):
                #make image preprocessing
                #1- flipping
                flip = iaa.Fliplr(1.0)
                
                #2- zooming
                zoom = iaa.Affine(scale = 1.3)
                
                #3- brightness
                brightness = iaa.Multiply((1, 1.3))
                
                #4- rotation
                rotation = iaa.Affine(rotate = (-25, 25))
                
                #perform data augmentation
                imageAugmentation = flip(image = image)
                imageAugmentation = brightness(image = imageAugmentation)
                imageAugmentation = zoom(image = imageAugmentation)
                imageAugmentation = rotation(image = imageAugmentation)
                
                #convert images to RGB colors
                RGBImage = cv2.cvtColor(imageAugmentation, cv2.COLOR_BGR2RGB)
                
                #resize
                resizedimage = cv2.resize(RGBImage, (imagesize, imagesize))
                
                #save image
                frames.append(resizedimage)
            count += 1
            
        else:
            break
    videocaptured.release()
    return frames

In [ ]:
#open data
Data = videos_directory + '/Real Life Violence Dataset'
print("there are {} violance videos and {} nonviolanve videos".format(len(os.listdir(Data + '/Violence')), len(os.listdir(Data + '/NonViolence'))))

In [ ]:
x = []
y = []
labels = ['Violence', 'NonViolence']

In [ ]:
for label in os.listdir(Data):
    videopath = os.path.join(Data, label)
    classnumber = labels.index(label)
    
    #take sample of 200 videos
    for i , video in enumerate(tqdm(os.listdir(videopath)[0:200])):
        videoframes = convert_video_to_frames(videopath + '/' + video)
        for j, frames in enumerate(videoframes):
            x.append(frames)
            y.append(classnumber)

#reshape x , y
x = np.array(x).reshape(-1, 128 *128 *3)
y = np.array(y)

In [ ]:
len(x)

In [ ]:
len(y)

# ***Spliting Data***

In [ ]:
#split data
sample = StratifiedShuffleSplit(n_splits = 2, test_size = 0.3, random_state = 42)

In [ ]:
for train, test in sample.split(x,y):
    x_train, x_test = x[train], x[test]
    y_train, y_test = y[train], y[test]
    
#reshape
x_train = x_train.reshape(-1, 128, 128, 3)/255
x_test = x_test.reshape(-1, 128, 128, 3)/255

#***ResNet50V2_model***

In [ ]:
def LoadModel():
    inputlayer = Input(shape =(128,128,3))
    model = ResNet50V2(weights = 'imagenet',
                      pooling = 'avg',
                      input_tensor = inputlayer)
    
    modelHead = model.output
    modelHead = Dense(1, activation = 'sigmoid')(modelHead)
    
    resnet_model = Model(inputs = model.input, outputs = modelHead)
    
    resnet_model.compile(loss = 'binary_crossentropy',
                        optimizer = 'adam',
                        metrics = ['accuracy'])
    return resnet_model
    

In [ ]:
from keras.models import load_model

resnet_model = LoadModel()

In [ ]:
resnet_model.summary()

# ***ploting _ ResNet50V2_Architecturesmodel***

In [ ]:
tf.keras.utils.plot_model(model, show_shapes=True)

# ***Trainin model***

In [ ]:
history = resnet_model.fit(x_train, y_train, epochs = 10, validation_data = (x_test, y_test),
                          batch_size = 128)

# ***VGG16 _model***

In [ ]:
def LoadModel():
    inputlayer = Input(shape=(128, 128, 3))
    model = VGG16(weights='imagenet',
                  pooling='avg',
                  include_top=False,
                  input_tensor=inputlayer)
    
    

    
    modelHead = model.output
    modelHead = Dense(1, activation='sigmoid')(modelHead)
    
    vgg_model = Model(inputs=model.input, outputs=modelHead)
    vgg_model.compile(loss = 'binary_crossentropy',
                        optimizer = 'adam',
                        metrics = ['accuracy'])
    
    return vgg_model

In [ ]:
vgg_model = LoadModel()

In [ ]:
vgg_model.summary()

# ***ploting VGg16 _Architectures_model***

In [ ]:
import tensorflow as tf
tf.keras.utils.plot_model(model, show_shapes=True)

# ***Training_model***

In [ ]:
history = vgg_model.fit(x_train, y_train, epochs = 40, validation_data = (x_test, y_test),
                          batch_size = 128)

# ***VGG19_model***

In [ ]:
def LoadModel():
    inputlayer = Input(shape=(128, 128, 3))
    model = VGG19(weights='imagenet',
                  pooling='avg',
                  include_top=False,
                  input_tensor=inputlayer)
    
    

    
    modelHead = model.output
    modelHead = Dense(1, activation='sigmoid')(modelHead)
    
    vgg_model = Model(inputs=model.input, outputs=modelHead)
    vgg_model.compile(loss = 'binary_crossentropy',
                        optimizer = 'adam',
                        metrics = ['accuracy'])
    
    return vgg_model

In [ ]:
vgg_model = LoadModel()

In [ ]:
vgg_model.summary()

# ***ploting VGg19 _Architectures_model***

In [ ]:
tf.keras.utils.plot_model(model, show_shapes=True)

# ***Training _model***

In [ ]:
history = vgg_model.fit(x_train, y_train, epochs = 20, validation_data = (x_test, y_test),
                          batch_size = 128)

# ***ResNet50 _model***

In [ ]:
def LoadModel():
    inputlayer = Input(shape =(128,128,3))
    model = ResNet50(weights = 'imagenet',
                      pooling = 'avg',
                      input_tensor = inputlayer)
    
    modelHead = model.output
    modelHead = Dense(1, activation = 'sigmoid')(modelHead)
    
    resnet_model = Model(inputs = model.input, outputs = modelHead)
    
    resnet_model.compile(loss = 'binary_crossentropy',
                        optimizer = 'adam',
                        metrics = ['accuracy'])
    return resnet_model

In [ ]:
resnet_model = LoadModel()

In [ ]:
resnet_model .summary()

 # ***ploting ResNet50 _Architectures_model***

In [ ]:
tf.keras.utils.plot_model(model, show_shapes=True)

# ***Training model***

In [ ]:
history = resnet_model.fit(x_train, y_train, epochs = 20, validation_data = (x_test, y_test),
                          batch_size = 128)



# ***If you liked this notebook, please do upvote.***

# ***This study was carried out by one novice who have just entered the world of deeplearning. Any questions, suggestions, criticism or feedback are highly appreciated.***
# ***Thank You!¶***
